# From CSV To Local DynamoDB
by <a href="https://www.linkedin.com/in/la-wun-nannda-b047681b5/">`La Wun Nannda`</a>

# Setup

In [1]:
# import libraries
import pandas as pd
import os
import boto3

# Setting up DynamoDB

Here are the official documentations-
- https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStartedDynamoDB.html
- https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/AccessingDynamoDB.html#Tools.CLI
- https://aws.amazon.com/cli/

# File Preparation: Formatting CSV

In DynamoDB, we must specify the primary key of the table once a table is created. The primary key is a unique identifier for each item in the table. There are two kinds:
1. simple primary key -> partition key
2. composite primary key -> partitation key + sort key 

## 1. Simple Primary Key
The DynamoDB uses the value of a partition key as an input to produce an output from the hash function. **No two items** can have the same partition key value.

## 2. Composite Primary Key
In the same way, a partition key outputs some result. But, this time, there can be **multiple items** with the same partition key values. The sort keys are used to sort items in order. With two keys, an item can be queried.

From previous notebooks, we can successfully export a file of either csv or excel. But our file contains entries with purely Burmese words which may or may not hinder the process of hashing. To ensure everything is ok, we will try to create unique hashed values on our own in the '.csv' file, using pandas. These values will be called `uuid`. The column can be used in our database as the simple primary key.

In [2]:
# before hashing
file = "../spreadsheets/BBC_webscraped_from_python.csv"
df = pd.read_csv(file)
df

,News Header,Time,Content
0,ဩဂုတ် ၁၀ ရက် နိုင်ငံတဝန်း သတင်းအနှစ်ချုပ် - သီ...,၁၀ သြဂုတ် ၂၀၂၄,ကျောက်မဲမြို့ နဲ့ လားရှိုးမြို့ကြားမှာရှိတဲ့ သ...
1,ကဖေးမြို့တော်- ကော်ဖီကြိုက်တဲ့ ရှန်ဟိုင်းမြို့,၇ သြဂုတ် ၂၀၂၄,ရှန်ဟိုင်းမြို့က လမ်းတွေပေါ် လျှောက်သွားရင် မြ...
2,အွန်လိုင်းရှာဖွေမှုကို ဂူဂဲလ် လက်ဝါးကြီးအုပ်မှ...,၆ သြဂုတ် ၂၀၂၄,အွန်လိုင်းပေါ် ရှာဖွေနိုင်မှုနဲ့ တခြားဆက်စပ် က...
3,နှစ်လနီးပါးတိုင်း တစ်ကြိမ်ဖြစ်နေတဲ့ ပြည်တွင်း ...,၄ သြဂုတ် ၂၀၂၄,ရန်ကုန်နဲ့ မန္တလေးမြို့ကြီးတွေပြင် ပြည်နယ်နဲ့တ...
4,တကောင်း နီကယ် စက်ရုံ ကို ပြည်သူ့ကာကွယ်ရေး လုံခ...,၂ သြဂုတ် ၂၀၂၄,စစ်ကိုင်းတိုင်း ထီးချိုင့်မြို့နယ်ထဲက တရုတ်ရင်...
...,...,...,...
672,ကိုဗစ်ကြောင့် ထိုင်းမှာ ပိတ်ထားတာတွေကို ပြန်ဖွင့်,၈ မေ ၂၀၂၀,ထိုင်းနိုင်ငံမှာ ကိုဗစ်၁၉ ကူးစက်မှုတွေများလာတဲ...
673,ရှမ်းပြည်က ပြောင်းဖူးတွေ မြစ်ထဲ ပစ်ချနေရတဲ့ နေ...,၈ မေ ၂၀၂၀,မြန်မာ-တရုတ်နယ်စပ်မှာ ပြောင်းဖူးသွားရောင်းတဲ့တ...
674,အိန္ဒိယမှာ ကိုရိုနာဗိုင်းရပ်စ် ကူးစက်မှု ထိုးတက်,၅ မေ ၂၀၂၀,အိန္ဒိယမှာ တစ်နေ့တည်း ကူးစက်မှု အများဆုံး အဖြစ...
675,ကိုဗစ်ကြောင့်တန်နဲ့ချီ ပျက်စီးခဲ့ရတဲ့ တရုတ်နယ်...,၂၈ ဧပြီ ၂၀၂၀,မြန်မာနိုင်ငံ ကနေ တရုတ်နိုင်ငံကို နယ်စပ်ကနေ တင...


In [3]:
# after hashing
df.insert(0, 'uuid', pd.util.hash_pandas_object(df).astype(str))
df

,uuid,News Header,Time,Content
0,4839253130577019882,ဩဂုတ် ၁၀ ရက် နိုင်ငံတဝန်း သတင်းအနှစ်ချုပ် - သီ...,၁၀ သြဂုတ် ၂၀၂၄,ကျောက်မဲမြို့ နဲ့ လားရှိုးမြို့ကြားမှာရှိတဲ့ သ...
1,10462253707508737678,ကဖေးမြို့တော်- ကော်ဖီကြိုက်တဲ့ ရှန်ဟိုင်းမြို့,၇ သြဂုတ် ၂၀၂၄,ရှန်ဟိုင်းမြို့က လမ်းတွေပေါ် လျှောက်သွားရင် မြ...
2,1201630294518368923,အွန်လိုင်းရှာဖွေမှုကို ဂူဂဲလ် လက်ဝါးကြီးအုပ်မှ...,၆ သြဂုတ် ၂၀၂၄,အွန်လိုင်းပေါ် ရှာဖွေနိုင်မှုနဲ့ တခြားဆက်စပ် က...
3,10103846380349943470,နှစ်လနီးပါးတိုင်း တစ်ကြိမ်ဖြစ်နေတဲ့ ပြည်တွင်း ...,၄ သြဂုတ် ၂၀၂၄,ရန်ကုန်နဲ့ မန္တလေးမြို့ကြီးတွေပြင် ပြည်နယ်နဲ့တ...
4,7617955021775301630,တကောင်း နီကယ် စက်ရုံ ကို ပြည်သူ့ကာကွယ်ရေး လုံခ...,၂ သြဂုတ် ၂၀၂၄,စစ်ကိုင်းတိုင်း ထီးချိုင့်မြို့နယ်ထဲက တရုတ်ရင်...
...,...,...,...,...
672,5412642120181965761,ကိုဗစ်ကြောင့် ထိုင်းမှာ ပိတ်ထားတာတွေကို ပြန်ဖွင့်,၈ မေ ၂၀၂၀,ထိုင်းနိုင်ငံမှာ ကိုဗစ်၁၉ ကူးစက်မှုတွေများလာတဲ...
673,2765255628281107808,ရှမ်းပြည်က ပြောင်းဖူးတွေ မြစ်ထဲ ပစ်ချနေရတဲ့ နေ...,၈ မေ ၂၀၂၀,မြန်မာ-တရုတ်နယ်စပ်မှာ ပြောင်းဖူးသွားရောင်းတဲ့တ...
674,4031870972622351436,အိန္ဒိယမှာ ကိုရိုနာဗိုင်းရပ်စ် ကူးစက်မှု ထိုးတက်,၅ မေ ၂၀၂၀,အိန္ဒိယမှာ တစ်နေ့တည်း ကူးစက်မှု အများဆုံး အဖြစ...
675,421296596469107945,ကိုဗစ်ကြောင့်တန်နဲ့ချီ ပျက်စီးခဲ့ရတဲ့ တရုတ်နယ်...,၂၈ ဧပြီ ၂၀၂၀,မြန်မာနိုင်ငံ ကနေ တရုတ်နိုင်ငံကို နယ်စပ်ကနေ တင...


# Make Function to Format CSV
We can see that our df now contains some numerical values. Now that we are ready to hash, we will prepare a function that outputs a new file with the '.csv' format. We will also rename the output file according to the original file to create consistency.

In [4]:
def format_csv(file): # directory to the file you want to format

    # Define a new file name and path
    file_name = os.path.basename(file)
    file_name_without_extension = file_name.split('.')[0]
    file_path = os.path.join('../spreadsheets/', file_name_without_extension + '_formatted.csv')

    # Read the csv file
    df = pd.read_csv(file)

    # Adding a new column to the left with a unique id created from hashing the dataframe
    df.insert(0, 'uuid', pd.util.hash_pandas_object(df).astype(str))
    
    # Make a new csv file
    df.to_csv(file_path, index=False)
    print(f'File "{file}" formatted and saved as "{file_path}"!')
    return file_path

In [5]:
# test
formatted_file = format_csv(file)

File "../spreadsheets/BBC_webscraped_from_python.csv" formatted and saved as "../spreadsheets/BBC_webscraped_from_python_formatted.csv"!


Now we are ready to input our 'csv' file to the DynamoDB. 

# Running Server
But first we must initiate a server. This can be done in the CLI with the AWS commands. The main takeaway is the we use `http://localhost:8000` to host our server. This notebook will not include developments, experiments, and tests that involves the CLI.

# Creating Table
Suppose we have hosted our server, we need to create an empty table first before we can put anything in it. For this task, we can avoid using AWS commands. The alternative is Python SDK from the AWS called `boto3` and pass several dictionaries.

Recall that we are using the simple primary key which uses the unique partition key. In *`KeySchema`*,
- The key type of the partition key is 'HASH' for our case.
- But if we use the composite primary key, the key type of the sort key is 'RANGE'.

In *`AttributeDefinitions`*, we define only the attributes that are part of the primary key (partition key and sort key). We can add other attributes to our items without defining them in *`AttributeDefinitions`*.

There are two kinds of table:
1. Provisioned: The number of read and write capacity units are defined. Billing is based on the provisioned capacity whether we use it fully or not. It is suitable with predictable traffic patterns and can reduce costs potententially in that scenario.
2. On-demand: There is no capacity unit. Billing is based on actual read and write requests.

For now, we will implement the provisioned mode. Thus, in *`ProvisionedThroughput`*, we define read and write capacity units for the DynamoDB table.
- ReadCapacityUnits: The number of strongly consistent reads per second expected to perform on the table. If we use eventually consistent reads, this number is doubled.
- WriteCapacityUnits: The number of writes per second expected to perform on the table.

In [6]:
# connect to db
dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000") # note that it is not 'https' but 'http'

# name the table
table_name = 'BBC-local-table'

try:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'uuid',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'uuid',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )

    # Wait until the table exists.
    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
    print(f"Table {table_name} created successfully.")
except Exception as e:
    print(f"Error creating table: {e}")

Error creating table: An error occurred (ResourceInUseException) when calling the CreateTable operation: Cannot create preexisting table


# Saving File in DynamoDB
Our table is prepared and we are ready to put items in it. In DynamoDB, the `float` data type is not supported directly. Instead, there is `Decimal` type from Python's `decimal` module to represent floating-point numbers. This ensures precision and compatibility with DynamoDB's number type. We can implement it as follows. Till now our data do not contain any float value.

In [7]:
df = pd.read_csv(formatted_file)
df

,uuid,News Header,Time,Content
0,4839253130577019882,ဩဂုတ် ၁၀ ရက် နိုင်ငံတဝန်း သတင်းအနှစ်ချုပ် - သီ...,၁၀ သြဂုတ် ၂၀၂၄,ကျောက်မဲမြို့ နဲ့ လားရှိုးမြို့ကြားမှာရှိတဲ့ သ...
1,10462253707508737678,ကဖေးမြို့တော်- ကော်ဖီကြိုက်တဲ့ ရှန်ဟိုင်းမြို့,၇ သြဂုတ် ၂၀၂၄,ရှန်ဟိုင်းမြို့က လမ်းတွေပေါ် လျှောက်သွားရင် မြ...
2,1201630294518368923,အွန်လိုင်းရှာဖွေမှုကို ဂူဂဲလ် လက်ဝါးကြီးအုပ်မှ...,၆ သြဂုတ် ၂၀၂၄,အွန်လိုင်းပေါ် ရှာဖွေနိုင်မှုနဲ့ တခြားဆက်စပ် က...
3,10103846380349943470,နှစ်လနီးပါးတိုင်း တစ်ကြိမ်ဖြစ်နေတဲ့ ပြည်တွင်း ...,၄ သြဂုတ် ၂၀၂၄,ရန်ကုန်နဲ့ မန္တလေးမြို့ကြီးတွေပြင် ပြည်နယ်နဲ့တ...
4,7617955021775301630,တကောင်း နီကယ် စက်ရုံ ကို ပြည်သူ့ကာကွယ်ရေး လုံခ...,၂ သြဂုတ် ၂၀၂၄,စစ်ကိုင်းတိုင်း ထီးချိုင့်မြို့နယ်ထဲက တရုတ်ရင်...
...,...,...,...,...
672,5412642120181965761,ကိုဗစ်ကြောင့် ထိုင်းမှာ ပိတ်ထားတာတွေကို ပြန်ဖွင့်,၈ မေ ၂၀၂၀,ထိုင်းနိုင်ငံမှာ ကိုဗစ်၁၉ ကူးစက်မှုတွေများလာတဲ...
673,2765255628281107808,ရှမ်းပြည်က ပြောင်းဖူးတွေ မြစ်ထဲ ပစ်ချနေရတဲ့ နေ...,၈ မေ ၂၀၂၀,မြန်မာ-တရုတ်နယ်စပ်မှာ ပြောင်းဖူးသွားရောင်းတဲ့တ...
674,4031870972622351436,အိန္ဒိယမှာ ကိုရိုနာဗိုင်းရပ်စ် ကူးစက်မှု ထိုးတက်,၅ မေ ၂၀၂၀,အိန္ဒိယမှာ တစ်နေ့တည်း ကူးစက်မှု အများဆုံး အဖြစ...
675,421296596469107945,ကိုဗစ်ကြောင့်တန်နဲ့ချီ ပျက်စီးခဲ့ရတဲ့ တရုတ်နယ်...,၂၈ ဧပြီ ၂၀၂၀,မြန်မာနိုင်ငံ ကနေ တရုတ်နိုင်ငံကို နယ်စပ်ကနေ တင...


In [8]:
# get table
table = dynamodb.Table(table_name)

# put csv into table
with table.batch_writer() as batch:
    for index, row in df.iterrows():
        item = row.to_dict() # convert row to dict
        item['uuid'] = str(item['uuid']) # change uuid to string
        batch.put_item(Item=item) # put item to db

print(f'The file "{formatted_file}" is exported to the DynamoDB table "{table_name}" successfully.')

The file "../spreadsheets/BBC_webscraped_from_python_formatted.csv" is exported to the DynamoDB table "BBC-local-table" successfully.


# Checking Out the Table
We should take some rows from our table to see if we have successfully put items into the table.

In [9]:
try:
    response = table.scan(Limit=2)
    items = response.get('Items', [])
    for item in items:
        print(item)
        print("")
except Exception as e:
    print(f"Error scanning table: {e}")

{'News Header': 'ဂျပန် ငွေကြေး ဖောင်းပွမှု', 'Time': '၁၀ ဇွန် ၂၀၂၂', 'uuid': '7450178344318034411', 'Content': 'လူနေမှု စရိတ် မြင့်တက်လာမှုက လူတွေ အားလုံးကို ထိခိုက်စေပါတယ်။ဆယ်စုနှစ်ပေါင်းများစွာ တည်ငြိမ်တဲ့စျေးနှုန်း၊ ဒါမှမဟုတ် ကျစျေးတွေနဲ့ ဝယ်ယူသုံးစွဲခဲ့ကြတာ ကျင့်သားရခဲ့ကြ တဲ့ ဂျပန်မှာ ငွေကြေးဖောင်းပွမှု နောက်ဆုံး ကြုံလာကြချိန်မှာ အကြီးအကျယ် တုန်လှုပ် ချောက်ချား စရာ ဖြစ်လာခဲ့ပါတယ်။တိုကျိုက ဘီဘီစီ သတင်းထောက် မာရီကို အွီး ပေးပို့တဲ့ သတင်းကို မညိုလဲ့ရည် ပြောပြပေးပါမယ်။'}

{'News Header': 'အစိုးရ စာချုပ်နဲ့ ထိုင်းမှာ အလုပ်လာလုပ်တဲ့ မြန်မာတစ်ထောင်ကျော် လပိုင်းအတွင်း အလုပ်ဖြုတ်ခံရ', 'Time': '၄ ဧပြီ ၂၀၂၃', 'uuid': '8915086772486097214', 'Content': 'ထိုင်းနိုင်ငံ စမွန်စခွန်ခရိုင်၊ မဟာချိုင်မှာရှိတဲ့ - အီလက်ထရောနစ်စက်ရုံက မြန်မာအလုပ် သမား ၁၀၆၀ ကျော်ဟာ ဧပြီ ၁ ရက်မှာ အလုပ်ထုတ်ခံလိုက်ရတယ်လို့ အထုတ်ခံရတဲ့အလုပ်သမားတွေနဲ့ အလုပ် သမားအရေးကူညီပေးနေသူတွေက ပြောပါတယ်။မြန်မာ-ထိုင်း နှစ်နိုင်ငံအစိုးရ သဘောတူညီမှု  နဲ့သွားတဲ့ ထိုင်းရောက်မြန်မာအလုပ်သမားတွေ အလုပ်ဖြုတ်ခံလိုက်ရတာဖြစ်ပြီး အများစုဟာ လုပ်သက်နည်းသူ

# Conclusion
The local DynamoDB should be set up and hosted before these procedures can run. In my experiment, it is easier to import a 'csv' file to the table than an excel spreadsheet. We can create individual modules for each of these steps and put together in a single file. Then, use `os` and `sys` functionalities to run the main script (including passing arguments) via the CLI.